<a href="https://colab.research.google.com/github/helmana/DLeeg/blob/master/model%20layers%20pop/random%20instance_2%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
>> Random instance Train Subject , New Subject

model3 -  1 layer pop

subject number = 109 >> 90 train subject  + 19 new subject 
channel number = 20 >> [21,23,29,31,33,35,36,40,8,10,12,41,46,48,50,52,54,60,61,62]

'''


'\n109 subject\n1 channel (Oz)\ntask: REO \nepoch: 60\nseed = 14\n\nTrainResult ([loss , acc])= [0.12535213132612147, 0.959957998996991]\nValidationResult ([loss , acc])= [0.1946041866179451, 0.9351468373493976]\nTestResult ([loss , acc])= [0.20246034180243572, 0.9319486027944112]\n\n'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install mne

     |████████████████████████████████| 6.4MB 2.6MB/s 


In [0]:
'''
Rivision:
980806:
  First version.
  
  

'''

'\nRivision:\n980806:\n  First version.\n  \n  \n\n'

In [0]:
# Parameters:

EpochNum=60
subject_numberT=109
RemoveBaseLine=0
Orthogonal=1
task_index = [1] # task: REO
list_channel=[21,23,29,31,33,35,36,40,8,10,12,41,46,48,50,52,54,60,61,62]
TryStr='Try1'
seed = 14

task_number=1
task_time = 60
sampel_number_per_sec =  160 # sampel rate
total_sampel_number =  sampel_number_per_sec *task_time # 60*160
#sample_shift = 4 #step len
inner_sample_shift = 4 # inner step len
outer_sample_shift =8  # outer step len
window_len= 20
batch_size=64

#Search_Space_Channel = [21,23,29,31,33,35,36,40,8,10,12,41,46,48,50,52,54,60,61,62]
#Search_Space_Channel_Name=["Fp1","Fp2","F7","F3","Fz","F4","F8","T7","C3","Cz","C4","T8","P7","P3","Pz","P4","P8","O1","Oz","O2"]


In [4]:
# For Visualization
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa

# General use
import itertools
from itertools import combinations 

import os.path
from scipy.spatial import distance
import numpy as np
import math
import random
import datetime
import time
import pytz
from time import gmtime, strftime

# For model training
from keras import layers
from keras import models
from keras import regularizers
from keras.utils import to_categorical
from keras import optimizers
from keras import backend as KerasBackend
from keras.models import load_model



# EEG Tools
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne import io
from mne import viz
from mne import Epochs, io, pick_types
from mne.event import define_target_events
from mne.time_frequency import psd_welch
print(__doc__)



Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/numba/decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


Automatically created module for IPython interactive environment


In [5]:
# load dataset in array
task_number= len(task_index)

# load dataset in array
list_raw_fnames = [[0]*task_number]*subject_numberT
for x in range(subject_numberT):
  list_raw_fnames[x] = mne.datasets.eegbci.load_data(x+1,task_index,path='/content/drive/My Drive/Deep Results/Database')

list_rawdata = np.zeros((subject_numberT,task_number), dtype='object')

for i in range(subject_numberT):
  for j in range(task_number):
    list_rawdata[i][j] = mne.io.read_raw_edf(list_raw_fnames[i][j], preload=True)


Do you want to set the path:
    /content/drive/My Drive/Deep Results/Database
as the default EEGBCI dataset path in the mne-python config [y]/n? y
Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Extracting EDF parameters from /content/drive/My Drive/Deep Results/Database/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /content/drive/My Drive/Deep Results/Database/MNE-eegbci-data/physiobank/database/eegmmidb/S002/S002R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /content/drive/My Drive/Deep Results/Database/MNE-eegbci-data/physiobank/database/eegmmidb/S003/S003R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.in

In [0]:
subject_number = 90
# new subject
new_subject =np.arange(subject_number, subject_numberT)

# img for each subject
subject_img_number = math.floor((total_sampel_number - sampel_number_per_sec - (window_len-1 )*inner_sample_shift) / outer_sample_shift) +1
np.random.seed(seed)  

subject_number_array_shuffled=np.arange(subject_number)
np.random.shuffle(subject_number_array_shuffled)

subject_img_number_array_shuffled=np.arange(subject_img_number)
np.random.shuffle(subject_img_number_array_shuffled)

train_index = len(subject_img_number_array_shuffled)//2
val_index = train_index + len(subject_img_number_array_shuffled)//4

train_shuff = subject_img_number_array_shuffled[:train_index]
val_shuff = subject_img_number_array_shuffled[train_index:val_index]
test_shuff = subject_img_number_array_shuffled[val_index:]


train_number = len(train_shuff)
val_number = len(val_shuff)
test_number = len(test_shuff)


In [0]:
def normalize_channel_data(ch , i, ch_min, ch_max):
  ch = ((ch - ch_min[i]) / (ch_max[i] - ch_min[i] ))
  return ch

def ProjectionVector(VecA,VecB):
    Projeted_VecB_on_VecA=np.dot(VecA,VecB)/np.dot(VecA,VecA)*VecA
    return Projeted_VecB_on_VecA

In [0]:
#################################################################################################
## Define General Data Generator
def GenerateGeneralData(BatchSize,ListRawDataMain,SubjectArrayShuffled,ListChannel,RemoveBaseLineFlag,OrthogonalFlag,ImageArrayShuffled,WindowLen,OuterSampleShift,InnerSampleShift,SampelNumberPerSec):
    
    Iter=0
    #ch_number=len(ListChannel)
    
    
    ##########################################################
    # Pre-Proceesing On Raw Data
    ListRawDataTemp=ListRawDataMain.copy()
    
    DataAllChannelsRawPerSubject, times =(ListRawDataTemp[0][0][:64,:9600])

    DataChannelsRaw   =np.zeros((len(SubjectArrayShuffled),ch_number,times.size),dtype = float)
    DataChannelsNormal=np.zeros((len(SubjectArrayShuffled),ch_number,times.size),dtype = float)
    DataChannelsOrt   =np.zeros((len(SubjectArrayShuffled),ch_number,times.size),dtype = float)
    e= 0    
    for s in SubjectArrayShuffled:

        DataAllChannelsRawPerSubject, times =(ListRawDataTemp[s][0][:64,:9600])
        
        # Extrac Channel T9 or T10 as Baseline
        DataChannelT9Raw =DataAllChannelsRawPerSubject[43]# Ch T9 (42) or T10 (43)


        ch_max =[]
        ch_min =[]
        # Extract Selected Channels+ Remove Baseline+ Normailze
        
        for i in range(len(ListChannel)):
            DataChannelsRaw[e][i]=DataAllChannelsRawPerSubject[ListChannel[i]].copy() -DataChannelT9Raw*RemoveBaseLineFlag

            ch_max = np.append(ch_max, max(DataChannelsRaw[e][i])) # max for each cannel
            ch_min = np.append(ch_min, min(DataChannelsRaw[e][i])) # min for each cannel

            DataChannelsNormal[e][i]=normalize_channel_data(DataChannelsRaw[e][i].copy(), i, ch_min, ch_max)
        e = e+1

        # Orthogonal Channel
        #DataChannelsOrt[e][0]=DataChannelsNormal[e][0]
        #DataChannelsOrt[e][1]=DataChannelsNormal[e][1]-ProjectionVector(DataChannelsNormal[e][0].copy(),DataChannelsNormal[e][1].copy())*OrthogonalFlag
        
        
        
    ##########################################################
    # Generate Data for Network
    while True:

      for j in ImageArrayShuffled:
          r = 0
          for s in SubjectArrayShuffled:

              if Iter ==0:
                  TrainImage=np.zeros((BatchSize, ch_number, WindowLen, SampelNumberPerSec), dtype = float)
                  TrainLabel=np.zeros((BatchSize),dtype=int)

              for i in range(len(ListChannel)):
                  for z in range (WindowLen):
                      ExtractedData = DataChannelsNormal[r][i][0+j*OuterSampleShift + z*InnerSampleShift:SampelNumberPerSec+ j*OuterSampleShift + z*InnerSampleShift]
                      TrainImage[Iter][i][z] = ExtractedData.copy()# Data
              r=r+1

              TrainLabel[Iter] = s # label

              Iter=Iter+1
              #Count=Count+1
              if Iter==BatchSize:
                Iter=0
                #print('Count=',Count)
                TrainLabelClass=to_categorical(TrainLabel,subject_numberT)
                TI = np.moveaxis(TrainImage, 1, -1)
                yield TI, TrainLabelClass 

                

In [0]:
ch_number=len(list_channel)
list_rawdataTemp=list_rawdata.copy()


In [13]:
model = load_model('/content/drive/My Drive/mymodels/model2_2.h5')
#model.summary()
model.pop()
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 20, 160, 64)       11584     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 10, 80, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 78, 128)        73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 4, 39, 128)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 2, 37, 256)        295168    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 1, 18, 256)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4608)             

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [0]:
batch_size = 1

In [0]:
def subject_Predict_generator(mymodel, subject_arr, img_arr ):
  sn = len(subject_arr)
  imgn = len(img_arr)
  #print("imgn", imgn)
  pre_GenTestData=GenerateGeneralData(BatchSize=batch_size,
                                  ListRawDataMain=list_rawdataTemp.copy(),
                                  SubjectArrayShuffled= subject_arr,
                                  ListChannel=list_channel,
                                  RemoveBaseLineFlag=RemoveBaseLine,
                                  OrthogonalFlag=Orthogonal,
                                  WindowLen=window_len,
                                  OuterSampleShift=outer_sample_shift,
                                  InnerSampleShift=inner_sample_shift,
                                  SampelNumberPerSec=sampel_number_per_sec,
                                  ImageArrayShuffled= img_arr)

  pre_Subject = mymodel.predict_generator(pre_GenTestData,steps=np.ceil(imgn*sn/batch_size))
  return pre_Subject


In [0]:
path = "/content/drive/My Drive/random dist"


In [0]:
#path = "/content/drive/My Drive/random dist"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Creation of the directory /content/drive/My Drive/random dist failed


In [50]:
##  random instance - Train Subject

base_subject = random.randint(0,subject_number-1)
base_rand_sample = random.randint(0,subject_img_number-1)
list_dist =[]

# subject 0 , instance 179
pre_S1 = subject_Predict_generator(model, [base_subject], [base_rand_sample])

for i in range (109 ):

  # subject i , random instance 
  pre_subject_arr = [i]
  rand_sample = [random.randint(0,subject_img_number-1)]

  ## Predict 2
  pre_S2 = subject_Predict_generator(model, pre_subject_arr, rand_sample)

  ##  Euclidea distance 
  d = distance.euclidean(pre_S1[0], pre_S2[0])
  #print(d)
  list_dist.append(d)
  
print(len(list_dist))
## Sort distance list
#list_dist.sort()

## Save in text file
textFileName = "textFile random dist_"+"base subject "+str(base_subject)+"_rand sample "+str(base_rand_sample)+".txt"
completeName = os.path.join(path, textFileName )

with open(completeName, 'w') as f:
  f.write( 'random dist:'  + '\n')
  f.write( 'base subject:'+str(base_subject) +' _ rand sample:'+str(base_rand_sample) + '\n'+ '\n')
  f.writelines(str(j) + '\n' for j in list_dist )
  
with open(completeName, 'r') as f:  
  print(f.readlines() )


109
['random dist:\n', 'base subject:82 _ rand sample:1063\n', '\n', '61.68320846557617\n', '81.07099914550781\n', '97.30227661132812\n', '133.13453674316406\n', '147.60841369628906\n', '58.542510986328125\n', '117.53657531738281\n', '90.21265411376953\n', '75.92208099365234\n', '57.11032485961914\n', '95.23693084716797\n', '94.118408203125\n', '73.1972427368164\n', '114.81141662597656\n', '60.03956985473633\n', '106.15522003173828\n', '98.05213928222656\n', '69.86250305175781\n', '98.11202239990234\n', '77.46245574951172\n', '72.23625946044922\n', '110.9220962524414\n', '76.21675872802734\n', '110.04949188232422\n', '95.23040771484375\n', '60.558658599853516\n', '174.1736602783203\n', '122.35267639160156\n', '180.07369995117188\n', '84.82441711425781\n', '98.42810821533203\n', '57.566192626953125\n', '53.355648040771484\n', '131.57691955566406\n', '190.90184020996094\n', '95.61370086669922\n', '113.86637115478516\n', '76.9766616821289\n', '110.51515197753906\n', '108.8543472290039\n',

In [51]:
##  random instance _ New Subject

base_subject = random.randint(90 ,108)
base_rand_sample = random.randint(0,subject_img_number-1)
list_dist =[]

# subject 0 , instance 179
pre_S1 = subject_Predict_generator(model, [base_subject], [base_rand_sample])

for i in range (109 ):

  # subject i , random instance 
  pre_subject_arr = [i]
  rand_sample = [random.randint(0,subject_img_number-1)]

  ## Predict 2
  pre_S2 = subject_Predict_generator(model, pre_subject_arr, rand_sample)

  ##  Euclidea distance 
  d = distance.euclidean(pre_S1[0], pre_S2[0])
  #print(d)
  list_dist.append(d)
  
print(len(list_dist))
## Sort distance list
#list_dist.sort()

## Save in text file
textFileName = "textFile random dist_"+"base subject "+str(base_subject)+"_rand sample "+str(base_rand_sample)+".txt"
completeName = os.path.join(path, textFileName )

with open(completeName, 'w') as f:
  f.write( 'random dist:'  + '\n')
  f.write( 'base subject:'+str(base_subject) +' _ rand sample:'+str(base_rand_sample) + '\n'+ '\n')
  f.writelines(str(j) + '\n' for j in list_dist )
  
with open(completeName, 'r') as f:  
  print(f.readlines() )


109
['random dist:\n', 'base subject:99 _ rand sample:799\n', '\n', '74.23799896240234\n', '88.97949981689453\n', '96.63225555419922\n', '98.55579376220703\n', '137.53573608398438\n', '75.09081268310547\n', '148.86773681640625\n', '65.29410552978516\n', '95.42191314697266\n', '127.95600128173828\n', '83.53624725341797\n', '91.52608489990234\n', '61.07000732421875\n', '95.16683959960938\n', '75.91883087158203\n', '75.11597442626953\n', '135.6522216796875\n', '85.48542022705078\n', '108.20515441894531\n', '100.1839370727539\n', '67.96511840820312\n', '64.43769836425781\n', '80.77601623535156\n', '126.626708984375\n', '74.86434173583984\n', '88.31514739990234\n', '97.08992767333984\n', '91.5192642211914\n', '152.13189697265625\n', '83.88471221923828\n', '102.83891296386719\n', '81.59580993652344\n', '68.3009262084961\n', '69.02494812011719\n', '199.6665802001953\n', '103.88789367675781\n', '96.39749908447266\n', '86.02212524414062\n', '78.10956573486328\n', '109.90775299072266\n', '95.438